In [ ]:
#| default_exp utils_email

# 📧 Email Sending

> Send emails via SMTP using markdown templates with markdown_merge.

In [ ]:
#| export

from __future__ import annotations
from typing import Optional, List, Dict, Any
import os
import logging
from pathlib import Path
from nbdev.showdoc import show_doc

logger = logging.getLogger(__name__)

## 🎯 Overview

| Category | Functions | Purpose |
|----------|-----------|---------|
| ⚙️ Config | `get_smtp_config` | Load SMTP settings from env |
| 📁 Templates | `get_template_path`, `load_template` | Manage markdown templates |
| ✉️ Sending | `send_email`, `send_batch_emails` | Core send functions |
| 🎁 Convenience | `send_welcome_email`, `send_invitation_email`, `send_password_reset_email` | Pre-built templates |

---

## 🏗️ Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                    Email Sending Flow                           │
├─────────────────────────────────────────────────────────────────┤
│  1. Load SMTP config from environment variables                 │
│  2. Load markdown template from templates/ directory            │
│  3. Substitute template variables (user_name, etc.)             │
│  4. Convert markdown to HTML via markdown_merge                 │
│  5. Send via SMTP (Azure, SendGrid, AWS SES, etc.)             │
└─────────────────────────────────────────────────────────────────┘
```

---

## 📋 Environment Variables

| Variable | Required | Default | Description |
|----------|----------|---------|-------------|
| `SMTP_HOST` | ✅ | - | SMTP server hostname |
| `SMTP_PORT` | ❌ | 587 | SMTP server port |
| `SMTP_USER` | ✅ | - | Auth username |
| `SMTP_PASSWORD` | ✅ | - | Auth password |
| `SMTP_MAIL_FROM` | ✅ | - | Sender email |
| `SMTP_STARTTLS` | ❌ | True | Use STARTTLS |
| `SMTP_SSL` | ❌ | False | Use SSL (disables TLS) |

## ⚙️ SMTP Configuration

| Function | Purpose |
|----------|---------|
| `get_smtp_config` | Load SMTP config from environment vars |

In [ ]:
#| export
def get_smtp_config() -> Dict[str, Any]:
    """Load SMTP configuration from environment variables."""
    smtp_host = os.getenv('SMTP_HOST')
    smtp_port = int(os.getenv('SMTP_PORT', '587'))
    smtp_user = os.getenv('SMTP_USER')
    smtp_pass = os.getenv('SMTP_PASSWORD')
    from_email = os.getenv('SMTP_MAIL_FROM')
    
    # SSL and TLS settings - check string and boolean values
    smtp_ssl = os.getenv('SMTP_SSL', 'False')
    use_ssl = smtp_ssl.lower() in ('true', '1', 'yes') if isinstance(smtp_ssl, str) else bool(smtp_ssl)
    
    smtp_starttls = os.getenv('SMTP_STARTTLS', 'True')
    use_tls = smtp_starttls.lower() in ('true', '1', 'yes') if isinstance(smtp_starttls, str) else bool(smtp_starttls)
    
    # If SSL is enabled, disable TLS
    if use_ssl:
        use_tls = False
    
    # Validate required fields
    if not all([smtp_host, smtp_user, smtp_pass, from_email]):
        raise ValueError(
            "SMTP_HOST, SMTP_USER, SMTP_PASSWORD, and SMTP_MAIL_FROM "
            "environment variables are required"
        )
    
    return {
        'host': smtp_host,
        'port': smtp_port,
        'user': smtp_user,
        'password': smtp_pass,
        'from_email': from_email,
        'from_name': '',  # Optional, can be added as SMTP_FROM_NAME if needed
        'use_ssl': use_ssl,
        'use_tls': use_tls
    }

In [ ]:
show_doc(get_smtp_config)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_email.py#L19){target="_blank" style="float:right; font-size:smaller"}

### get_smtp_config

>      get_smtp_config ()

*Load SMTP configuration from environment variables.*

## 📁 Template Management

| Function | Purpose |
|----------|---------|
| `get_template_path` | Resolve path to template file |
| `load_template` | Read template content as string |

In [ ]:
#| export
def get_template_path(template_name: str) -> Path:
    """Get absolute path to email template file."""
    # Get the package directory
    package_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()
    template_path = package_dir / 'templates' / f'{template_name}.md'
    
    if not template_path.exists():
        raise FileNotFoundError(f"Template not found: {template_path}")
    
    return template_path


def load_template(template_name: str) -> str:
    """Load markdown email template as string."""
    template_path = get_template_path(template_name)
    return template_path.read_text(encoding='utf-8')

In [ ]:
show_doc(load_template)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_email.py#L105){target="_blank" style="float:right; font-size:smaller"}

### load_template

>      load_template (template_name:str)

*Load markdown email template as string.*

In [ ]:
show_doc(get_template_path)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_email.py#L86){target="_blank" style="float:right; font-size:smaller"}

### get_template_path

>      get_template_path (template_name:str)

*Get absolute path to email template file.*

## ✉️ Email Sending

| Function | Purpose |
|----------|---------|
| `send_email` | Send single email with template |
| `send_batch_emails` | Send to multiple recipients |

In [ ]:
#| export
def send_email(
    to_email: str,
    to_name: str,
    subject: str,
    template_name: str,
    template_vars: Dict[str, str],
    test: bool = False,
    smtp_config: Optional[Dict[str, Any]] = None
) -> Dict[str, Any]:
    """Send single email using markdown template with variable substitution."""
    try:
        from markdown_merge import MarkdownMerge, get_addr
    except ImportError:
        raise ImportError(
            "markdown_merge not installed. Install with: pip install markdown_merge"
        )
    
    # Load SMTP config
    if smtp_config is None:
        config = get_smtp_config()
    else:
        config = smtp_config
    
    # Prepare SMTP config for markdown_merge
    mm_smtp_cfg = {
        'host': config['host'],
        'port': config['port'],
        'user': config['user'],
        'password': config['password'],
        'use_ssl': config.get('use_ssl', False),
        'use_tls': config.get('use_tls', True)
    }
    
    # Load template
    template = load_template(template_name)
    
    # Prepare addresses
    from_addr = get_addr(config['from_email'], config.get('from_name', ''))
    to_addr = get_addr(to_email, to_name)
    
    # Create MarkdownMerge instance
    mm = MarkdownMerge(
        [to_addr],
        from_addr,
        subject,
        template,
        smtp_cfg=mm_smtp_cfg,
        inserts=[template_vars],
        test=test
    )
    
    # Send email
    try:
        mm.send_msgs()
        
        if test:
            logger.info(f"Test email printed for {to_email}")
            return {
                "status": "test",
                "to_email": to_email,
                "message": "Email printed (test mode)"
            }
        else:
            logger.info(f"Email sent successfully to {to_email}")
            return {
                "status": "success",
                "to_email": to_email,
                "subject": subject
            }
    except Exception as e:
        logger.error(f"Failed to send email to {to_email}: {str(e)}")
        return {
            "status": "error",
            "to_email": to_email,
            "error": str(e)
        }

In [ ]:
show_doc(send_email)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_email.py#L118){target="_blank" style="float:right; font-size:smaller"}

### send_email

>      send_email (to_email:str, to_name:str, subject:str, template_name:str,
>                  template_vars:Dict[str,str], test:bool=False,
>                  smtp_config:Optional[Dict[str,Any]]=None)

*Send single email using markdown template with variable substitution.*

## 📦 Batch Sending

| Function | Purpose |
|----------|---------|
| `send_batch_emails` | Send personalized emails to multiple recipients |

In [ ]:
#| export
def send_batch_emails(
    recipients: List[Dict[str, str]],
    subject: str,
    template_name: str,
    template_vars_list: List[Dict[str, str]],
    test: bool = False,
    pause: float = 0.2,
    smtp_config: Optional[Dict[str, Any]] = None
) -> List[Dict[str, Any]]:
    """Send personalized emails to multiple recipients with per-recipient variables."""
    try:
        from markdown_merge import MarkdownMerge, get_addr
    except ImportError:
        raise ImportError(
            "markdown_merge not installed. Install with: pip install markdown_merge"
        )
    
    # Load SMTP config
    if smtp_config is None:
        config = get_smtp_config()
    else:
        config = smtp_config
    
    # Prepare SMTP config for markdown_merge
    mm_smtp_cfg = {
        'host': config['host'],
        'port': config['port'],
        'user': config['user'],
        'password': config['password'],
        'use_ssl': config.get('use_ssl', False),
        'use_tls': config.get('use_tls', True)
    }
    
    # Load template
    template = load_template(template_name)
    
    # Prepare addresses
    from_addr = get_addr(config['from_email'], config.get('from_name', ''))
    to_addrs = [get_addr(r['email'], r['name']) for r in recipients]
    
    # Create MarkdownMerge instance
    mm = MarkdownMerge(
        to_addrs,
        from_addr,
        subject,
        template,
        smtp_cfg=mm_smtp_cfg,
        inserts=template_vars_list,
        test=test
    )
    
    # Send emails
    results = []
    try:
        mm.send_msgs(pause=pause)
        
        for recipient in recipients:
            if test:
                results.append({
                    "status": "test",
                    "to_email": recipient['email'],
                    "message": "Email printed (test mode)"
                })
            else:
                results.append({
                    "status": "success",
                    "to_email": recipient['email'],
                    "subject": subject
                })
        
        logger.info(f"Sent {len(recipients)} emails successfully")
    except Exception as e:
        logger.error(f"Batch send failed: {str(e)}")
        for recipient in recipients:
            results.append({
                "status": "error",
                "to_email": recipient['email'],
                "error": str(e)
            })
    
    return results

In [ ]:
show_doc(send_batch_emails)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_email.py#L223){target="_blank" style="float:right; font-size:smaller"}

### send_batch_emails

>      send_batch_emails (recipients:List[Dict[str,str]], subject:str,
>                         template_name:str,
>                         template_vars_list:List[Dict[str,str]],
>                         test:bool=False, pause:float=0.2,
>                         smtp_config:Optional[Dict[str,Any]]=None)

*Send personalized emails to multiple recipients with per-recipient variables.*

## 🎁 Convenience Functions

Pre-built functions for common email types:

| Function | Template | Purpose |
|----------|----------|---------|
| `send_welcome_email` | `welcome.md` | New user onboarding |
| `send_invitation_email` | `invitation.md` | Invite to tenant |
| `send_password_reset_email` | `password_reset.md` | Password recovery |

In [ ]:
#| export
def send_welcome_email(
    to_email: str,
    to_name: str,
    user_name: str,
    tenant_name: str,
    dashboard_url: str,
    test: bool = False
) -> Dict[str, Any]:
    """Send welcome email to new user with tenant dashboard link."""
    return send_email(
        to_email=to_email,
        to_name=to_name,
        subject=f"Welcome to {tenant_name}!",
        template_name='welcome',
        template_vars={
            'user_name': user_name,
            'tenant_name': tenant_name,
            'dashboard_url': dashboard_url,
            'to_email': to_email
        },
        test=test
    )

In [ ]:
show_doc(send_welcome_email)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_email.py#L333){target="_blank" style="float:right; font-size:smaller"}

### send_welcome_email

>      send_welcome_email (to_email:str, to_name:str, user_name:str,
>                          tenant_name:str, dashboard_url:str, test:bool=False)

*Send welcome email to new user with tenant dashboard link.*

In [ ]:
#| export
def send_invitation_email(
    to_email: str,
    to_name: str,
    inviter_name: str,
    tenant_name: str,
    invitation_url: str,
    test: bool = False
) -> Dict[str, Any]:
    """Send invitation email with link to join tenant."""
    return send_email(
        to_email=to_email,
        to_name=to_name,
        subject=f"{inviter_name} invited you to join {tenant_name}",
        template_name='invitation',
        template_vars={
            'inviter_name': inviter_name,
            'tenant_name': tenant_name,
            'invitation_url': invitation_url,
            'to_email': to_email
        },
        test=test
    )

In [ ]:
show_doc(send_invitation_email)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_email.py#L369){target="_blank" style="float:right; font-size:smaller"}

### send_invitation_email

>      send_invitation_email (to_email:str, to_name:str, inviter_name:str,
>                             tenant_name:str, invitation_url:str,
>                             test:bool=False)

*Send invitation email with link to join tenant.*

In [ ]:
#| export
def send_password_reset_email(
    to_email: str,
    to_name: str,
    user_name: str,
    reset_url: str,
    test: bool = False
) -> Dict[str, Any]:
    """Send password reset email with secure reset link."""
    return send_email(
        to_email=to_email,
        to_name=to_name,
        subject="Reset Your Password",
        template_name='password_reset',
        template_vars={
            'user_name': user_name,
            'reset_url': reset_url,
            'to_email': to_email
        },
        test=test
    )

In [ ]:
show_doc(send_password_reset_email)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_email.py#L405){target="_blank" style="float:right; font-size:smaller"}

### send_password_reset_email

>      send_password_reset_email (to_email:str, to_name:str, user_name:str,
>                                 reset_url:str, test:bool=False)

*Send password reset email with secure reset link.*